In [4]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [5]:
from AIToolbox.torchtrain.train_loop import TrainLoop, TrainLoopModelCheckpointEndSave
from AIToolbox.torchtrain.model import ModelWrap
from AIToolbox.torchtrain.data.batch_model_feed_defs import AbstractModelFeedDefinition
from AIToolbox.torchtrain.callbacks.performance_eval_callbacks import ModelPerformanceEvaluation, ModelPerformancePrintReport
from AIToolbox.experiment.result_package.basic_packages import ClassificationResultPackage

## Define the model

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Define the batch feed def

In [24]:
class MNISTModelFeedDefinition(AbstractModelFeedDefinition):
    def get_loss(self, model, batch_data, criterion, device):
        data, target = batch_data
        data, target = data.to(device), target.to(device)

        output = model(data)
        loss = criterion(output, target)

        return loss
    
    def get_loss_eval(self, model, batch_data, criterion, device):
        return self.get_loss(model, batch_data, criterion, device)

    def get_predictions(self, model, batch_data, device):
        data, y_test = batch_data
        data = data.to(device)

        output = model(data)
        y_pred = output.argmax(dim=1, keepdim=False)  # get the index of the max log-probability

        return y_test, y_pred.cpu(), {}

In [10]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [12]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=100, shuffle=True, **kwargs)

In [13]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=100, shuffle=True, **kwargs)

In [15]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
criterion = F.nll_loss

In [26]:
callbacks = [ModelPerformanceEvaluation(ClassificationResultPackage(), {'batch_size': 100, 'lr': 0.001},
                                        on_train_data=True, on_val_data=True),
             ModelPerformancePrintReport(['train_Accuracy', 'val_Accuracy'], strict_metric_reporting=True)]

In [27]:
TrainLoop(ModelWrap(model, MNISTModelFeedDefinition()),
          train_loader, test_loader, None,
          optimizer, criterion)(num_epoch=3, callbacks=callbacks)



  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 2/600 [00:00<00:41, 14.50it/s]



Epoch: 0




  1%|          | 4/600 [00:00<00:40, 14.68it/s]

  1%|          | 6/600 [00:00<00:39, 15.05it/s]

  1%|▏         | 8/600 [00:00<00:38, 15.48it/s]

  2%|▏         | 10/600 [00:00<00:37, 15.66it/s]

  2%|▏         | 12/600 [00:00<00:38, 15.46it/s]

  2%|▏         | 14/600 [00:00<00:37, 15.65it/s]

  3%|▎         | 16/600 [00:01<00:37, 15.77it/s]

  3%|▎         | 18/600 [00:01<00:36, 15.80it/s]

  3%|▎         | 20/600 [00:01<00:37, 15.67it/s]

  4%|▎         | 22/600 [00:01<00:36, 15.73it/s]

  4%|▍         | 24/600 [00:01<00:36, 15.81it/s]

  4%|▍         | 26/600 [00:01<00:36, 15.92it/s]

  5%|▍         | 28/600 [00:01<00:35, 16.04it/s]

  5%|▌         | 30/600 [00:01<00:36, 15.42it/s]

  5%|▌         | 32/600 [00:02<00:36, 15.52it/s]

  6%|▌         | 34/600 [00:02<00:36, 15.68it/s]

  6%|▌         | 36/600 [00:02<00:35, 15.82it/s]

  6%|▋         | 38/600 [00:02<00:36, 15.59it/s]

  7%|▋         | 40/600 [00:02<00:35, 15.78it/s]

  7%|▋         | 42/600 [00:02<00:35, 15.90it/s]

 

 49%|████▊     | 292/600 [00:19<00:20, 15.30it/s]

 49%|████▉     | 294/600 [00:19<00:25, 12.07it/s]

 49%|████▉     | 296/600 [00:19<00:26, 11.43it/s]

 50%|████▉     | 298/600 [00:19<00:25, 11.88it/s]

 50%|█████     | 300/600 [00:20<00:28, 10.43it/s]

 50%|█████     | 302/600 [00:20<00:27, 10.87it/s]

 51%|█████     | 304/600 [00:20<00:26, 11.06it/s]

 51%|█████     | 306/600 [00:20<00:24, 11.80it/s]

 51%|█████▏    | 308/600 [00:20<00:22, 12.94it/s]

 52%|█████▏    | 310/600 [00:20<00:21, 13.56it/s]

 52%|█████▏    | 312/600 [00:20<00:20, 14.05it/s]

 52%|█████▏    | 314/600 [00:21<00:19, 14.74it/s]

 53%|█████▎    | 316/600 [00:21<00:18, 15.52it/s]

 53%|█████▎    | 318/600 [00:21<00:17, 15.72it/s]

 53%|█████▎    | 320/600 [00:21<00:17, 15.90it/s]

 54%|█████▎    | 322/600 [00:21<00:17, 16.23it/s]

 54%|█████▍    | 324/600 [00:21<00:16, 16.40it/s]

 54%|█████▍    | 326/600 [00:21<00:16, 16.57it/s]

 55%|█████▍    | 328/600 [00:21<00:16, 16.77it/s]

 55%|█████▌    | 330/600 [00:22

 96%|█████████▋| 578/600 [00:36<00:01, 16.85it/s]

 97%|█████████▋| 580/600 [00:37<00:01, 16.85it/s]

 97%|█████████▋| 582/600 [00:37<00:01, 16.70it/s]

 97%|█████████▋| 584/600 [00:37<00:00, 16.87it/s]

 98%|█████████▊| 586/600 [00:37<00:00, 16.91it/s]

 98%|█████████▊| 588/600 [00:37<00:00, 16.89it/s]

 98%|█████████▊| 590/600 [00:37<00:00, 17.00it/s]

 99%|█████████▊| 592/600 [00:37<00:00, 16.93it/s]

 99%|█████████▉| 594/600 [00:37<00:00, 17.02it/s]

 99%|█████████▉| 596/600 [00:37<00:00, 17.07it/s]

100%|█████████▉| 598/600 [00:38<00:00, 17.03it/s]

100%|██████████| 600/600 [00:38<00:00, 16.96it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  1%|          | 4/600 [00:00<00:16, 35.57it/s]

...

 96%|█████████▌| 576/600 [00:17<00:00, 34.36it/s]

 97%|█████████▋| 580/600 [00:17<00:00, 34.47it/s]

 97%|█████████▋| 584/600 [00:17<00:00, 34.71it/s]

 98%|█████████▊| 588/600 [00:17<00:00, 34.19it/s]

 99%|█████████▊| 592/600 [00:17<00:00, 34.62it/s]

 99%|█████████▉| 596/600 [00:17<00:00, 34.86it/s]

100%|██████████| 600/600 [00:17<00:00, 34.87it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  4%|▍         | 4/100 [00:00<00:02, 34.30it/s]

Auto purging prediction store




  8%|▊         | 8/100 [00:00<00:02, 34.41it/s]

 12%|█▏        | 12/100 [00:00<00:02, 34.35it/s]

 16%|█▌        | 16/100 [00:00<00:02, 34.28it/s]

 20%|██        | 20/100 [00:00<00:02, 34.13it/s]

 24%|██▍       | 24/100 [00:00<00:02, 33.61it/s]

 28%|██▊       | 28/100 [00:00<00:02, 32.84it/s]

 32%|███▏      | 32/100 [00:00<00:02, 32.59it/s]

 36%|███▌      | 36/100 [00:01<00:01, 33.48it/s]

 40%|████      | 40/100 [00:01<00:01, 33.50it/s]

 44%|████▍     | 44/100 [00:01<00:01, 33.41it/s]

 48%|████▊     | 48/100 [00:01<00:01, 33.80it/s]

 52%|█████▏    | 52/100 [00:01<00:01, 34.40it/s]

 56%|█████▌    | 56/100 [00:01<00:01, 34.91it/s]

 60%|██████    | 60/100 [00:01<00:01, 35.09it/s]

 64%|██████▍   | 64/100 [00:01<00:01, 35.34it/s]

 68%|██████▊   | 68/100 [00:01<00:00, 34.57it/s]

 72%|███████▏  | 72/100 [00:02<00:00, 34.33it/s]

 76%|███████▌  | 76/100 [00:02<00:00, 34.41it/s]

 80%|████████  | 80/100 [00:02<00:00, 32.88it/s]

 84%|████████▍ | 84/100 [00:02<00:00, 29.31it/s]


------------------  End of epoch performance report  -------------------
train_Accuracy: 0.9535
val_Accuracy: 0.9576
Getting train set predictions from store
Getting validation set predictions from store
-----------------  End of training performance report  -----------------
train_end_train_Accuracy: 0.9535
train_end_val_Accuracy: 0.9576


Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

## Experiment tracking training

In [28]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
criterion = F.nll_loss

In [29]:
callbacks = [ModelPerformanceEvaluation(ClassificationResultPackage(), {'batch_size': 100, 'lr': 0.001},
                                        on_train_data=True, on_val_data=True),
             ModelPerformancePrintReport(['train_Accuracy', 'val_Accuracy'], strict_metric_reporting=True)]

In [31]:
TrainLoopModelCheckpointEndSave(ModelWrap(model, MNISTModelFeedDefinition()),
                                train_loader, test_loader, test_loader,
                                optimizer, criterion,
                                project_name='localRunCNNTest',
                                experiment_name='CNN_MNIST_test',
                                local_model_result_folder_path='model_results',
                                hyperparams={'batch_size': 100, 'lr': 0.001},
                                test_result_package=ClassificationResultPackage())(num_epoch=3, callbacks=callbacks)



  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 2/600 [00:00<00:41, 14.25it/s]



Epoch: 0




  1%|          | 4/600 [00:00<00:40, 14.63it/s]

  1%|          | 6/600 [00:00<00:39, 15.15it/s]

  1%|▏         | 8/600 [00:00<00:38, 15.52it/s]

  2%|▏         | 10/600 [00:00<00:37, 15.73it/s]

  2%|▏         | 12/600 [00:00<00:36, 15.92it/s]

  2%|▏         | 14/600 [00:00<00:36, 15.88it/s]

  3%|▎         | 16/600 [00:01<00:36, 15.93it/s]

  3%|▎         | 18/600 [00:01<00:36, 15.79it/s]

  3%|▎         | 20/600 [00:01<00:37, 15.62it/s]

  4%|▎         | 22/600 [00:01<00:36, 15.75it/s]

  4%|▍         | 24/600 [00:01<00:36, 15.70it/s]

  4%|▍         | 26/600 [00:01<00:38, 14.86it/s]

  5%|▍         | 28/600 [00:01<00:38, 15.03it/s]

  5%|▌         | 30/600 [00:01<00:37, 15.19it/s]

  5%|▌         | 32/600 [00:02<00:38, 14.92it/s]

  6%|▌         | 34/600 [00:02<00:37, 14.93it/s]

  6%|▌         | 36/600 [00:02<00:37, 14.98it/s]

  6%|▋         | 38/600 [00:02<00:37, 15.16it/s]

  7%|▋         | 40/600 [00:02<00:36, 15.23it/s]

  7%|▋         | 42/600 [00:02<00:36, 15.17it/s]

 

 49%|████▊     | 292/600 [00:18<00:17, 17.23it/s]

 49%|████▉     | 294/600 [00:18<00:18, 16.66it/s]

 49%|████▉     | 296/600 [00:18<00:18, 16.67it/s]

 50%|████▉     | 298/600 [00:18<00:18, 16.73it/s]

 50%|█████     | 300/600 [00:18<00:17, 16.94it/s]

 50%|█████     | 302/600 [00:18<00:17, 16.98it/s]

 51%|█████     | 304/600 [00:18<00:17, 17.08it/s]

 51%|█████     | 306/600 [00:19<00:17, 17.22it/s]

 51%|█████▏    | 308/600 [00:19<00:17, 17.00it/s]

 52%|█████▏    | 310/600 [00:19<00:16, 17.10it/s]

 52%|█████▏    | 312/600 [00:19<00:16, 17.18it/s]

 52%|█████▏    | 314/600 [00:19<00:16, 17.25it/s]

 53%|█████▎    | 316/600 [00:19<00:16, 17.35it/s]

 53%|█████▎    | 318/600 [00:19<00:16, 17.36it/s]

 53%|█████▎    | 320/600 [00:19<00:16, 17.27it/s]

 54%|█████▎    | 322/600 [00:20<00:16, 17.28it/s]

 54%|█████▍    | 324/600 [00:20<00:15, 17.30it/s]

 54%|█████▍    | 326/600 [00:20<00:15, 17.17it/s]

 55%|█████▍    | 328/600 [00:20<00:15, 17.12it/s]

 55%|█████▌    | 330/600 [00:20

 96%|█████████▋| 578/600 [00:35<00:01, 16.81it/s]

 97%|█████████▋| 580/600 [00:36<00:01, 16.92it/s]

 97%|█████████▋| 582/600 [00:36<00:01, 16.95it/s]

 97%|█████████▋| 584/600 [00:36<00:00, 16.99it/s]

 98%|█████████▊| 586/600 [00:36<00:00, 16.95it/s]

 98%|█████████▊| 588/600 [00:36<00:00, 16.85it/s]

 98%|█████████▊| 590/600 [00:36<00:00, 16.92it/s]

 99%|█████████▊| 592/600 [00:36<00:00, 17.06it/s]

 99%|█████████▉| 594/600 [00:36<00:00, 16.87it/s]

 99%|█████████▉| 596/600 [00:37<00:00, 16.24it/s]

100%|█████████▉| 598/600 [00:37<00:00, 16.08it/s]

100%|██████████| 600/600 [00:37<00:00, 16.02it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  1%|          | 5/600 [00:00<00:14, 41.52it/s]

...



  2%|▏         | 10/600 [00:00<00:14, 41.24it/s]

  2%|▏         | 13/600 [00:00<00:16, 35.98it/s]

  3%|▎         | 16/600 [00:00<00:17, 33.77it/s]

  3%|▎         | 20/600 [00:00<00:17, 34.12it/s]

  4%|▍         | 24/600 [00:00<00:16, 34.11it/s]

  5%|▍         | 28/600 [00:00<00:16, 34.37it/s]

  5%|▌         | 32/600 [00:00<00:16, 33.54it/s]

  6%|▌         | 36/600 [00:01<00:16, 34.99it/s]

  7%|▋         | 40/600 [00:01<00:15, 35.95it/s]

  7%|▋         | 44/600 [00:01<00:15, 36.50it/s]

  8%|▊         | 48/600 [00:01<00:14, 36.90it/s]

  9%|▊         | 52/600 [00:01<00:14, 37.00it/s]

  9%|▉         | 56/600 [00:01<00:14, 37.34it/s]

 10%|█         | 60/600 [00:01<00:14, 37.65it/s]

 11%|█         | 64/600 [00:01<00:14, 37.33it/s]

 11%|█▏        | 68/600 [00:01<00:14, 36.85it/s]

 12%|█▏        | 72/600 [00:01<00:14, 36.61it/s]

 13%|█▎        | 76/600 [00:02<00:14, 36.47it/s]

 13%|█▎        | 80/600 [00:02<00:14, 36.94it/s]

 14%|█▍        | 84/600 [00:02<00:14, 36.76it/s]

TRAIN LOSS: 0.39679430122176806




  8%|▊         | 8/100 [00:00<00:02, 33.47it/s]

 12%|█▏        | 12/100 [00:00<00:02, 34.57it/s]

 16%|█▌        | 16/100 [00:00<00:02, 36.00it/s]

 20%|██        | 20/100 [00:00<00:02, 36.30it/s]

 24%|██▍       | 24/100 [00:00<00:02, 36.85it/s]

 28%|██▊       | 28/100 [00:00<00:01, 37.67it/s]

 33%|███▎      | 33/100 [00:00<00:01, 38.45it/s]

 37%|███▋      | 37/100 [00:00<00:01, 38.39it/s]

 41%|████      | 41/100 [00:01<00:01, 37.59it/s]

 45%|████▌     | 45/100 [00:01<00:01, 37.10it/s]

 49%|████▉     | 49/100 [00:01<00:01, 37.11it/s]

 53%|█████▎    | 53/100 [00:01<00:01, 37.52it/s]

 57%|█████▋    | 57/100 [00:01<00:01, 38.09it/s]

 61%|██████    | 61/100 [00:01<00:01, 37.75it/s]

 65%|██████▌   | 65/100 [00:01<00:00, 37.46it/s]

 69%|██████▉   | 69/100 [00:01<00:00, 37.81it/s]

 74%|███████▍  | 74/100 [00:01<00:00, 38.64it/s]

 78%|███████▊  | 78/100 [00:02<00:00, 38.69it/s]

 82%|████████▏ | 82/100 [00:02<00:00, 38.64it/s]

 86%|████████▌ | 86/100 [00:02<00:00, 38.77it/s]


VAL LOSS: 0.37770835891366006


FileNotFoundError: [Errno 2] No such file or directory: '<ipython-input-31-6e3f288e0d40>'